In [2]:
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()
import dense_correspondence
reload(dense_correspondence)
from dense_correspondence.training.training import *
import sys
import logging
import json
import time

#utils.set_default_cuda_visible_devices()
# utils.set_cuda_visible_devices([0]) # use this to manually set CUDA_VISIBLE_DEVICES

import dense_correspondence.dataset.spartan_dataset_masked
reload(dense_correspondence.dataset.spartan_dataset_masked)
import dense_correspondence.training.training
reload(dense_correspondence.training.training)
from dense_correspondence.training.training import DenseCorrespondenceTraining
from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset, OPDataSelector
logging.basicConfig(level=logging.INFO)

import dense_correspondence.evaluation.evaluation
reload(dense_correspondence.evaluation.evaluation)
from dense_correspondence.evaluation.evaluation import DenseCorrespondenceEvaluation

# Multi Object Pursuit
utils.add_object_pursuit_to_python_path()
import object_pursuit
reload(object_pursuit)
from dense_correspondence.training.op_training import OPDenseCorrespondenceTraining

from dense_correspondence.network.op_dense_correspondence_network import \
    OPMultiDenseCorrespondenceNetwork, OPMultiDenseCorrespondenceNetworkV2

cuda = torch.cuda.is_available()
DEVICE = "cuda:1" if cuda else "cpu"
if cuda:
    print("CUDA GPU!")
else:
    print("CPU!")

CUDA GPU!


## Load the configuration for training

In [ ]:
# Dataset
verbose = False
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', 'op_pretraining.yaml')
data_config = utils.getDictFromYamlFilename(config_filename)
data_config['logs_root_path'] = '/home/ashek/code/data/pdc/logs_proto'

# Training
train_config_file = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'training', 'op_pretraining.yaml')
train_config = utils.getDictFromYamlFilename(train_config_file)
train_config['device'] = DEVICE
train_config['dense_correspondence_network']['device'] = DEVICE

logging_dir = "/home/ashek/code/data/pdc/trained_models/tutorials"
num_iterations = 3500
d = 3 # the descriptor dimension
datetime_str = time.strftime('%Y_%m_%d_%H:%M:%S', time.localtime())
name = "op_multinet_pretraining_" + datetime_str
train_config["training"]["logging_dir_name"] = name
train_config["training"]["logging_dir"] = logging_dir
train_config["dense_correspondence_network"]["descriptor_dimension"] = d
train_config["training"]["num_iterations"] = num_iterations
print(json.dumps(train_config, indent=2))

## Train the network

This should take about ~12-15 minutes with a GTX 1080 Ti

In [ ]:
# dataset.get_pose_data(dataset.get_scene_list()[0])
data_config

In [ ]:
# All of the saved data for this network will be located in the
# code/data/pdc/trained_models/tutorials/caterpillar_3 folder
print(train_config.keys())
print "training descriptor of dimension %d" %(d)
NetworkClass = OPMultiDenseCorrespondenceNetworkV2
# NetworkClass = OPMultiDenseCorrespondenceNetwork
train_config["dense_correspondence_network"]["model_class"] = str(NetworkClass)
train = OPMultiDenseCorrespondenceTraining(NetworkClass=NetworkClass, config=train_config)
train.load_dataset_from_config(config=data_config)
train.run()
print "finished training descriptor of dimension %d" %(d)

# Each batch, select a random image from a random scene
# for that image, find another image from the same scene
# but a taken from a diff view.
# then generate both pixel matches and non-matches 
# yes there is a heuristic approach, but it seeems complex
# and thus runtime is long. Prune non-matches based on:
# 1. depth is 0 (thus invalid, cannot use)
# 2. Project pixels from im1 to onto im2 and check if outside FOV
# 3. check if pixel is occluded using some method
# complicated... don't touch

# Fine-tune on New Objects

In [ ]:
# New objects
# Datasets available:
# caterpillar_upright
# mugs
# shoes
# drills
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', 'finetune.yaml')
finetune_config = utils.getDictFromYamlFilename(config_filename)
finetune_config['logs_root_path'] = '/home/ashek/code/data/pdc/logs_proto'
finetune_dataset = SpartanDataset(config=finetune_config)
print(json.dumps(finetune_config, indent=2))

In [ ]:
num_iterations = 1500
d = 3 # the descriptor dimension
train_config["training"]["num_iterations"] = num_iterations
print(json.dumps(train_config, indent=2))

In [ ]:
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_data_relative_path_to_absolute_path(model_folder)
print("Loading %s" % model_folder)
iteration = 3501
print "training descriptor of dimension %d" %(d)
finetune = DenseCorrespondenceTraining(dataset=finetune_dataset, config=train_config)
finetune.run_from_pretrained(model_folder, iteration)
print "finished training descriptor of dimension %d" %(d)

In [ ]:
!ls /home/ashek/code/data/pdc/trained_models/tutorials/caterpillar_3

## Evaluate the network quantitatively

This should take ~5 minutes.

In [ ]:
# Evaluate initial model (no fine-tuning) on original train dataset
DCE = DenseCorrespondenceEvaluation
iteration = 3501
num_image_pairs = 100
save_folder_name = "analysis"
DCE.run_evaluation_on_network(model_folder, iteration=iteration, num_image_pairs=num_image_pairs,
                             dataset=dataset, save_folder_name=save_folder_name)      

In [ ]:
# Evaluate fine-tuned model on original train dataset
iteration = 5002
num_image_pairs = 100
save_folder_name = "finetune_analysis_orig_data"
DCE.run_evaluation_on_network(model_folder, iteration=iteration, num_image_pairs=num_image_pairs,
                             dataset=dataset, save_folder_name=save_folder_name)         

In [ ]:
# Evaluate fine-tuned model on new finetuned dataset
DCE = DenseCorrespondenceEvaluation
iteration = 5002
num_image_pairs = 100
save_folder_name = "finetune_analysis_finetune_data"
DCE.run_evaluation_on_network(model_folder, iteration=iteration, num_image_pairs=num_image_pairs,
                             dataset=finetune_dataset, save_folder_name=save_folder_name)      

See `evaluation_quantitative_tutorial.ipynb` for a better place to display the plots.

In [ ]:
# Finetune on 2nd additional dataset just to emphasize issue further
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', 'finetune2.yaml')
finetune2_config = utils.getDictFromYamlFilename(config_filename)
finetune2_config['logs_root_path'] = '/home/ashek/code/data/pdc/logs_proto'
finetune2_dataset = SpartanDataset(config=finetune_config)
print(json.dumps(finetune2_config, indent=2))

In [ ]:
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_data_relative_path_to_absolute_path(model_folder)
print("Loading %s" % model_folder)
iteration = 5002
print "training descriptor of dimension %d" %(d)
finetune2 = DenseCorrespondenceTraining(dataset=finetune2_dataset, config=train_config)
finetune2.run_from_pretrained(model_folder, iteration)
print "finished training descriptor of dimension %d" %(d)

In [ ]:
# Evaluate fine-tuned2 model on new finetuned2 dataset
DCE = DenseCorrespondenceEvaluation
iteration = 6503
num_image_pairs = 100
save_folder_name = "finetune2_analysis_finetune_data2"
DCE.run_evaluation_on_network(model_folder, iteration=iteration, num_image_pairs=num_image_pairs,
                             dataset=finetune2_dataset, save_folder_name=save_folder_name)      

In [ ]:
# Evaluate fine-tuned model on original train dataset
iteration = 6503
num_image_pairs = 100
save_folder_name = "finetune2_analysis_orig_data"
DCE.run_evaluation_on_network(model_folder, iteration=iteration, num_image_pairs=num_image_pairs,
                             dataset=dataset, save_folder_name=save_folder_name)         